# Cylinder Surface Sandpile

The notebook below takes a 2D texture and wraps it around a 3D cylinder frame. If **rotate == True** the cylinder will rotate 360 degrees take a video frame at every turn.

In [1]:
from vpython import *
scene = canvas()
scene.width = scene.height = 960

rotate = False
filenom = 'images/Cylinder_LineX_03drops_100k_01.png'

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Create snapshot button and attached function.

In [2]:
from datetime import datetime
import math

def SnapshotButton(b):
    date_time = datetime.now().strftime("%m_%d_%Y_%H_%M_%S")
    scene.capture(f'{filenom.replace(".png", "")}_{date_time}')

snapButton = button(text='Take Snapshot', bind=SnapshotButton, name='snapshot')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

This code was the original method for mapping the 2D texture onto the cylinder, but the texture repeats every 180 degrees instead of wrapping fully around the cylinder once.

In [3]:
#rod = cylinder(pos=vector(0,-(h/2.0),0), axis=vector(0,h,0), radius=r, texture={'file': filenom, 'place': 'all', 'turn': 1})
#rod

Load texture, calculate key values needed to generate coordinates, define texture segment dimensions.

In [4]:
from PIL import Image

img = Image.open(filenom)
w, h = img.size
ratio = h / w
print(f'Width: {w}px, Height: {h}px, Ratio: {ratio:.5f}')

numTxSegments = 30
numTySegments = int(numTxSegments * ratio)

r = w / (2.0 * math.pi)
deltaRads = (2.0 * math.pi) / numTxSegments
print(f'Radius: {r:.5f}, Delta Radians: {deltaRads:.5f}')

x0 = r * math.cos(0.0)
x1 = r * math.cos(deltaRads)

y0 = r * math.sin(0.0)
y1 = r * math.sin(deltaRads)

segW = math.sqrt(math.pow(x1 - x0, 2) + math.pow(y1 - y0, 2))
segH = h / numTySegments

print(f'Num Tx Segments: {numTxSegments}, Segment Width: {segW:.5f}')
print(f'Num Ty Segments: {numTySegments}, Segment Height: {segH:.5f}')

Width: 1440px, Height: 1448px, Ratio: 1.00556
Radius: 229.18312, Delta Radians: 0.20944
Num Tx Segments: 30, Segment Width: 47.91232
Num Ty Segments: 30, Segment Height: 48.26667


Render cylinder quadrants while mapping sections of the 2D texture.

In [5]:
startY = -h / 2.0
verts = []

deltaTx = 1.0 / numTxSegments
deltaTy = 1.0 / numTySegments
for j in range(numTySegments + 1):
    verts.append([])
    for i in range(numTxSegments + 1): # Switched y and z
        x = r * math.cos(i * deltaRads)
        y = startY + (j * segH)
        z = r * math.sin(i * deltaRads)
        
        tx = i * deltaTx
        ty = j * deltaTy
        verts[j].append(vertex(pos=vector(x,y,z), texpos=vector(tx,ty,0))) # Not sure what to specify for 'normal' yet
        
for j in range(numTySegments):
    for i in range(numTxSegments):
        quad(vs=[verts[j][i], verts[j][i+1], verts[j+1][i+1], verts[j+1][i]], texture={'file': filenom})

Rotate the cylinder while saving out frames along the way if **rotate == True**.

In [6]:
numSegments = 120
deltaAngle = (2.0 * math.pi) / numSegments
ctr = 1

# https://www.glowscript.org/#/user/GlowScriptDemos/folder/Examples/program/RotatingPoints/edit
if rotate == True:
    sleep(5)
    while rotate == True:
        # Currently there isn't a way to rotate a points object, so rotate scene.forward:
        sleep(2)
        scene.forward = scene.forward.rotate(angle=-deltaAngle, axis=vec(0,1,0)) # Angle is in radians
        scene.capture(f'{filenom.replace(".png", "")}_{ctr:05}')
        
        ctr = ctr + 1
        if ctr > (numSegments+1):
            rotate = False